<h1>Data refining</h1>
<p>This script contains functionality to refine data and produce a new data set. The module <i>pandas</i> contains many built-in functions for this task which was capitalised on.</p>

In [1]:
import pandas as pd

In [2]:
import refine as rf

<p>Data, stored in a CSV file, was provided to help us get started. The data file is read in below.</p>

In [3]:
df=pd.read_csv("../ACometLandingTwitterArchive.csv")

<p>Firstly, the function <i>drop_duplicates()</i> was used so that if any two rows are identical in the data set, they are recognised and the copy is removed. </p>

In [4]:
#Number of Tweets: Length of original data set excluding header
print(len(df)-1)

77318


In [5]:
df = df.drop_duplicates(keep = False)

In [6]:
#Number of Tweets: Length of original data set excluding header after removing duplicates
print(len(df)-1)

77298


<p>Secondly, the user's language settings (<i>user_lang</i>) are looped over. All the values in this column were changed to lowercase, so that “en-gb” and “en-GB” are recognised as the same.<br/> If the language is null but the rest of the row contains reasonable values, there is no point in discarding the data, and therefore <i>user_lang</i> will be set to “en” by default. If the rest of the data is damaged, later functions will remove it.</p>

In [16]:
#Number of unique users
len(df['from_user'].unique())

49924

In [35]:
#Number of unique users
s = df['from_user'].groupby(df['from_user_id_str'].astype(int)).value_counts().nlargest(100)
print (s)

from_user_id_str  from_user      
2851551783        weruqc89bc72       116
356021744         gurumescudi         97
32427002          spaceflashnews      88
1401527256        misola23a           81
1591816866        canuhandlme         73
                                    ... 
2856653680        installiphone_      23
2858503108        computermemory_     23
14846573          Sondas              22
69860028          masanorimusic       22
449749984         Koln                22
Name: from_user, Length: 100, dtype: int64


In [7]:
#Unique column values before, note the presesnce of 'nan' and
#unregulated mix of lower and upper case
df["user_lang"].unique()

array(['en', 'fr', 'de', 'th', 'es', 'it', 'en-gb', 'ca', 'ru', 'ar',
       'pt', 'nl', 'tr', 'ja', 'no', 'pl', 'ko', 'da', 'ro', 'uk', 'fi',
       'sv', 'hu', 'cs', 'id', 'zh-cn', 'fil', 'xx-lc', 'zh-Hans', 'el',
       'es-MX', 'zh-CN', 'en-AU', 'eu', 'en-GB', 'pt-PT', 'gl', 'he',
       'bg', nan, 'zh-tw', 'nb', 'fa', 'vi', 'msa', 'ur', 'hi'],
      dtype=object)

In [8]:
df = rf.langCheck(df)

In [9]:
##Unique column values after langCheck
df["user_lang"].unique()

array(['en', 'fr', 'de', 'th', 'es', 'it', 'en-gb', 'ca', 'ru', 'ar',
       'pt', 'nl', 'tr', 'ja', 'no', 'pl', 'ko', 'da', 'ro', 'uk', 'fi',
       'sv', 'hu', 'cs', 'id', 'zh-cn', 'fil', 'xx-lc', 'zh-hans', 'el',
       'es-mx', 'en-au', 'eu', 'pt-pt', 'gl', 'he', 'bg', 'zh-tw', 'nb',
       'fa', 'vi', 'msa', 'ur', 'hi'], dtype=object)

<p>Next, the <i>removeNAN()</i> function checks and removes rows of missing data. It uses the functions <i>isnull()</i> and <i>dropna()</i> to remove NaN values (NaN is used by <i>pandas</i> to represent nulls). For the cases when null represents over 50% of a columns data (such as the reply column), these values are considered valid and not removed.</p>

In [10]:
#Length of data set before removing null values
print(len(df))

77299


In [11]:
df = rf.removeNAN(df)

In [12]:
#Length of data set after removing null values
print(len(df))

77083


<p>Additional functions were written to check for other anomalies in the data. Usernames are checked for both length and for special characters (only letters, numerals and “_” are allowed).<br/> The text column is also checked for length, as it cannot be over 140 characters. There are no cases of such corruption in the provided data set, but it may prove useful when validating other, less clean data. </p>

In [13]:
# df = rf.fromUserCheck(df)
df = rf.textCheck(df)

<p>We did not add format checking for all of the columns as that would have been tedious work, and from our observation, when the data was damaged, it usually meant that its values were null rather than malformed. <br/>
Once the data set has been adequately refined, it is moved to a new file.
</p>

In [14]:
rf.makeCSV(df)

In [15]:
df

,id_str,from_user,text,created_at,time,geo_coordinates,user_lang,in_reply_to_user_id_str,in_reply_to_screen_name,from_user_id_str,in_reply_to_status_id_str,source,profile_image_url,user_followers_count,user_friends_count,status_url,entities_str
0,5.409300e+17,amika0078788556,RT @VersaTechnology: Congratulations @Philae20...,Fri Dec 05 18:07:14 +0000 2014,05/12/2014 18:07,NaN,en,NaN,NaN,1.297570e+09,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/3555068293...,425.0,113.0,http://twitter.com/amika0078788556/statuses/54...,"{""hashtags"":[{""text"":""Philae"",""indices"":[49,56..."
1,5.409300e+17,ChrisDMarshall,CometWatch 2 December Â» Rosetta - ESA's comet...,Fri Dec 05 18:05:46 +0000 2014,05/12/2014 18:05,NaN,en,NaN,NaN,4.220121e+07,NaN,"<a href=""https://dev.twitter.com/docs/tfw"" rel...",http://pbs.twimg.com/profile_images/228087980/...,347.0,1286.0,http://twitter.com/ChrisDMarshall/statuses/540...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
2,5.409300e+17,MHuuskoL,RT @EUCouncil: After the #CometLanding - Astro...,Fri Dec 05 18:05:36 +0000 2014,05/12/2014 18:05,NaN,en,NaN,NaN,5.744348e+08,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://pbs.twimg.com/profile_images/4973510588...,215.0,310.0,http://twitter.com/MHuuskoL/statuses/540929998...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
3,5.409290e+17,SaraGomezAranci,RT @EUCouncil: After the #CometLanding - Astro...,Fri Dec 05 18:03:00 +0000 2014,05/12/2014 18:03,NaN,fr,NaN,NaN,1.057356e+09,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/5391705804...,741.0,310.0,http://twitter.com/SaraGomezAranci/statuses/54...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
4,5.409290e+17,CBCDay6,RT @shaunmajumder: Feels good to be the @CBCDa...,Fri Dec 05 18:02:32 +0000 2014,05/12/2014 18:02,NaN,en,NaN,NaN,1.772769e+08,NaN,"<a href=""http://twitter.com/download/android"" ...",http://pbs.twimg.com/profile_images/4590694303...,4331.0,2098.0,http://twitter.com/CBCDay6/statuses/5409292290...,"{""hashtags"":[{""text"":""MiniMansbridge"",""indices..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77314,5.324600e+17,ABForScience,This means that the actual landing will be ar...,Wed Nov 12 09:09:26 +0000 2014,12/11/2014 09:09,NaN,en,NaN,NaN,2.333123e+09,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",http://pbs.twimg.com/profile_images/4320695593...,155.0,212.0,http://twitter.com/ABForScience/statuses/53246...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77315,5.324600e+17,atieyK,RT @ObservingSpace: Weâ€™ve been waiting 10 ye...,Wed Nov 12 09:09:26 +0000 2014,12/11/2014 09:09,NaN,en,NaN,NaN,5.830372e+07,NaN,"<a href=""https://twitter.com/download/android""...",http://pbs.twimg.com/profile_images/5057311637...,354.0,1088.0,http://twitter.com/atieyK/statuses/53246014845...,"{""hashtags"":[{""text"":""cometlanding"",""indices"":..."
77316,5.324600e+17,j0nny5,RT @maxplanckpress: Accomazzo (flight director...,Wed Nov 12 09:09:26 +0000 2014,12/11/2014 09:09,NaN,en,NaN,NaN,1.791486e+07,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://pbs.twimg.com/profile_images/89991673/j...,92.0,63.0,http://twitter.com/j0nny5/statuses/53246014825...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
77317,5.324600e+17,nsentse,7 hours of waiting #CometLanding,Wed Nov 12 09:09:26 +0000 2014,12/11/2014 09:09,NaN,en,NaN,NaN,2.043400e+07,NaN,"<a href=""http://twitter.com/download/iphone"" r...",http://pbs.twimg.com/profile_images/3693655850...,200.0,267.0,http://twitter.com/nsentse/statuses/5324601482...,"{""hashtags"":[{""text"":""CometLanding"",""indices"":..."
